In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 Import the libraries

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import WordPunctTokenizer
import os
import urllib.request
import tensorflow as tf
import torch
import os
import torchvision
import tarfile
from torch.utils.data import random_split
from torchvision.datasets.utils import download_url
import matplotlib.pyplot as plt
import torch.nn as nn
import spacy
from torch.utils.data import DataLoader

In [ ]:
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
pip install openpyxl

In [ ]:
train=pd.read_excel('../input/enterpret-absa/train.xlsx')
test=pd.read_excel('../input/enterpret-absa/test.xlsx')

In [ ]:
def parse_data(data):
    for i in range(len(data)):
        if(type(data.loc[i,'aspect'])==float):
            data.loc[i,'aspect']=str(data.loc[i,'aspect'])
    data['text_tok'] = data['text'].apply(lambda x: x.lower())
    data['text_tok'] = data['text_tok'].apply(custom_tokenize)
    data['aspect_tok'] = data['aspect'].apply(lambda x: x.lower())
    data['aspect_tok'] = data['aspect_tok'].apply(custom_tokenize)
    return data

def custom_tokenize(text):#split sentences into a list of word
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(text)
    words = [word for word in tokens if word.isalnum()]
    return words

def max_len(data):#function to find maximum length of text and aspect
    max_text=0
    max_asp=0
    for i in range(len(data)):
        max_text=max(max_text,len(data.loc[i,'text_tok']))
        max_asp=max(max_asp,len(data.loc[i,'aspect_tok']))
    return max_text,max_asp

def find_w(data_train_1,data_test_1):#function to create word to indices dictionary
    word2idx={}
    i=0
    for sentence in data_train_1['text_tok']:
        for word in sentence:
            if word not in word2idx.values():
                word2idx[i]= word
                i=i+1
    
    for sentence in data_train_1['aspect_tok']:
        for word in sentence:
            if word not in word2idx.values():
                word2idx[i]= word
                i=i+1
                
                
    for sentence in data_test_1['text_tok']:
        for word in sentence:
            if word not in word2idx.values():
                word2idx[i]= word
                i=i+1
                
    for sentence in data_test_1['aspect_tok']:
        for word in sentence:
            if word not in word2idx.values():
                word2idx[i]= word
                i=i+1
    return word2idx
    


def create_vocab(data_train_1):
    word2idx = {}
    
    for sentence in data_train_1['text_tok']:
        for word in sentence:
            if word not in word2idx:
                word2idx[word] = len(word2idx)
    
    idx2word ={v: k for k, v in word2idx.items()}
    return word2idx,idx2word


def create_dict(vocab,embed):#function to create dictionary
    max_len=len(vocab)
    w_ind={}
    i_w={}
    print(max_len)
    for i in range(len(vocab)):
        if vocab[i] not in w_ind:
            w_ind[vocab[i]]=len(w_ind)+1
            i_w[i+1]=vocab[i]
    
    return   w_ind,i_w    


In [ ]:
train_2=train.copy()
data=parse_data(train_2)

test_2=test.copy()
data_test=parse_data(test_2)
#print(data)
max_tex,max_asp=max_len(data)
vocab=find_w(data,data_test)
w_ind,i_w=create_dict(vocab,300)

In [ ]:
#save vocabulary in a json file for future use in inference and test set evaluation
a_file = open("full_vocabulary.json", "w")
json.dump(vocab, a_file)
a_file.close()

In [ ]:
glove = pd.read_csv('../input/glove-data/glove.6B.300d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove_embedding = {key: val.values for key, val in glove.T.items()}

In [ ]:
emb_mat=np.zeros((len(vocab)+1,300))#matrix of glove embeddings of all words in test and train sets

In [ ]:
emb_mat.shape

In [ ]:
emb_mat[0]

In [ ]:
for i in range(len(i_w)):
    i_word = i_w[i+1]
    if(i_word not in glove_embedding.keys()):
        emb_mat[i+1]=np.random.rand(300)
    else:
        emb_mat[i+1]=glove_embedding[i_word]
    #vocab.append(i_word)
    #embeddings.append(i_embeddings)

In [ ]:
tex=np.empty([len(data),max_tex])
aspec=np.empty([len(data),max_asp])
for i in range(len(data)):#create a index encoding of all sentences in train set
    sent=data.loc[i,'text_tok']
    asp=data.loc[i,'aspect_tok']
    encoded_sentences = np.array([w_ind[word] for word in sent])
    encoded_sen=np.append(encoded_sentences,np.zeros(278-len(encoded_sentences)))
    encoded_asp=np.array([w_ind[word] for word in asp])
    encoded_as=np.append(encoded_asp,np.zeros(8-len(encoded_asp)))
    tex[i]=encoded_sen
    aspec[i]=encoded_as
    
    

In [ ]:
tex=torch.LongTensor(tex)

In [ ]:
aspec=torch.LongTensor(aspec)

In [ ]:
data['label']=data['label'].apply(lambda x:int(x))

In [ ]:
labels= torch.tensor(data['label'].values)

In [ ]:
tex.shape

Creating train and validation sets.Since data is almost at random ,we can directly do splitting without using any sampler.

In [ ]:
train_tex=tex[:3200]
train_aspec=aspec[:3200]
train_labels=labels[:3200]

In [ ]:
val_tex=tex[3200:]
val_aspec=aspec[3200:]
val_labels=labels[3200:]

In [ ]:
train_ds=TensorDataset(train_tex,train_aspec,train_labels)
batch_size = 20
train_dl = DataLoader(train_ds, batch_size, shuffle=False)#creating dataloaders

In [ ]:
val_ds=TensorDataset(val_tex,val_aspec,val_labels)
batch_size = 20
val_dl = DataLoader(val_ds, batch_size, shuffle=False)

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device=get_default_device()

In [ ]:
class aspectClassificationBase(nn.Module):#utility class for training and testing.Needs to be present
                          #in all notebooks since it is inherited by models.

    def training_step(self,batch):
        model.train()
        text,aspect,labels=batch
        out=self(text,aspect,20)
        #labels=int(labels)
        labels=labels.to(torch.long)
        loss=F.cross_entropy(out,labels)
        return loss
    
    def validation_step(self,batch):
        model.eval()
        text,aspect,labels=batch
        out=self(text,aspect,20)
        labels=labels.to(torch.long)
        loss=F.cross_entropy(out,labels)
        acc=accuracy(labels,out)
        return {'val_loss':loss.detach(),'val_acc':acc}
    
    def validation_epoch_end(self,result):
        loss=[x['val_loss'] for x in result]
        acc= [x['val_acc'] for x in result]
        l_b=torch.stack(loss).mean()
        a_b=torch.stack(acc).mean()
        return {'val_loss':l_b.item(), 'val_acc': a_b.item()}
    
    def epoch_end(self,epoch,result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch,result['val_loss'], result['val_acc']))

In [ ]:
class LSTM_AOA(aspectClassificationBase):
    def __init__(self,emb_mat,num_classes,embedding_dim,vocab_size_t,vocab_size_a):
        super(LSTM_AOA,self).__init__()
        self.embedding_dim = embedding_dim
        self.vocab_size_t = vocab_size_t
        self.vocab_size_a=vocab_size_a
        self.polarities=num_classes
        self.embed = nn.Embedding.from_pretrained(emb_mat,freeze=True)
        self.lstm =nn.LSTM(self.embedding_dim,128,batch_first=True,num_layers=1,bidirectional=True)
        #self.lstm_a  =nn.LSTM(self.embedding_dim,128,batch_first=True,bidirectional=True)
        self.dense = nn.Linear(256,self.polarities)
        self.softmax_c=nn.Softmax(dim=2)
        self.softmax_r=nn.Softmax(dim=1)
        self.softmax=nn.Softmax(dim=1)

    def forward(self, text,aspect,batch_size):
        #asp_raw_indices=inputs[:,278:,:]
       # y1=inputs[0][278:][:]
        #y2=inputs[1][278:][:]
        
        #text_raw_indices = torch.stack([x1,x2])
        #asp_raw_indices=torch.stack([y1,y2])
        #print(asp_raw_indices.shape)
        #print(text_raw_indices.shape)
        x= self.embed(text)
        x,_=self.lstm(x)
       # print(x.shape)
        
        x2=self.embed(aspect)
        x2,_=self.lstm(x2)
        #print(x2.shape)
        x2=x2.permute(0,2,1)
        
        x3=torch.matmul(x,x2)
        #print(x3.shape)
        x4=x3.detach().clone()
        x5=x3.detach().clone()
        x5=self.softmax_c(x5)
        #print(x5.shape)
        x4=self.softmax_r(x4)
        #x4=x4.permute(0,2,1)
        x5=x5.sum(dim=1)/self.vocab_size_t
        #print(x5.shape)
        #print(x5)
        x5=x5.reshape(batch_size,self.vocab_size_a,1)
        x6=torch.matmul(x4,x5)
     
       # print(x6.shape)
       # print(x6.shape)
        x=x.permute(0,2,1)
       # print(x.shape)
        x7=torch.matmul(x,x6)
        #print(x7.shape)
        
        
        x7=x7.reshape(batch_size,-1)
        #print(x7.shape)
        x7=self.dense(x7)
        #print(x7.shape)
        #out=self.softmax(x7)
        #x_len = torch.sum(text_raw_indices != 0, dim=-1)
        #_, (h_n, _) = self.lstm(x, x_len)
        #out = self.dense(h_n[0])
        return x7

In [ ]:
emb_mat_t=torch.tensor(emb_mat)

In [ ]:
embedding_dim=300
number_classes=3

In [ ]:
model =LSTM_AOA(emb_mat_t,number_classes,embedding_dim,max_tex,max_asp)#creating instance of model class
model.double()
model


In [ ]:
device

In [ ]:
train_dl=DeviceDataLoader(train_dl,device)
val_dl=DeviceDataLoader(val_dl,device)#transporting dataloaders to device

In [ ]:
to_device(model, device)#model to device

In [ ]:
model.parameters

In [ ]:
def accuracy(act,pred):
    _, preds = torch.max(pred, dim=1)
    #print(preds[0])
    #print(act[0])
    return torch.tensor(torch.sum(preds==act).item() / len(preds))

In [ ]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    optimizer = opt_func(model.parameters(), lr)
    history = [] # for recording epoch-wise results
    
    for epoch in range(epochs):
        
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        # Validation phase
        with torch.no_grad():
            result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
fit(10, 0.01, model, train_dl, val_dl)

In [ ]:
fit(10, 0.005, model, train_dl, val_dl)

In [ ]:
fit(10, 0.002, model, train_dl, val_dl)

In [ ]:
torch.save(model, 'aoa_b20_e30_entire_model.pth')


In [ ]:
torch.save(model.state_dict(), 'aoa_b20_nodrop_e30_state_dict.pth')
